In [24]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import joblib 
import re

from sklearn.base import TransformerMixin, BaseEstimator
class Transformer(TransformerMixin,BaseEstimator):
    def __init__(self):
        self.wt=[]
        self.ram=[]
        self.x_res=[]
        self.y_res=[]
        self.all_res=[]
        self.scrn_type=[]
        self.cpu_freq=[]
        self.cpu_type=[]
    def fit(self,dataset,y=None):
        self.dataset=dataset
        for i in list(dataset.columns):
            X=dataset[i]
            if i=='Weight':
                for j in X:
                    self.wt.append(float(j.split("kg")[0]))
        
            elif i=="Ram":
                for j in X:
                    self.ram.append(int(j.split("GB")[0]))
            
            
    
            elif i=="ScreenResolution":
                most_freq=X.value_counts().keys()[0]
                for j in X.values:
                    temp=re.search(r"(\d{3,4}x\d{3,4})",j).group()
                    temp2=j.split(temp)
                    temp=temp.split('x')
                    if temp2[0]=='':
                        self.scrn_type.append(most_freq)
                        self.x_res.append(int(temp[0]))
                        self.y_res.append(int(temp[1]))
                        self.all_res.append([most_freq,int(temp[0]),int(temp[1])])
                    else:
                        self.scrn_type.append(temp2[0])
                        self.x_res.append(int(temp[0]))
                        self.y_res.append(int(temp[1]))
                        self.all_res.append([temp2[0],int(temp[0]),int(temp[1])])
            
            elif i=="Cpu":
                for j in X.values:
                    temp=re.search(r'(\d{1}.\d{0,2}GHz)',j)
                    if temp:
                        temp0=re.split(r'(\d{1}.\d{0,2}GHz)',j)[0]
                        temp=temp.group().split('GHz')[0]
                        if re.search(r'(\d \d)',temp):
                            temp=temp.split(" ")[1]
                            self.cpu_freq.append(float(temp))
                        else:
                            self.cpu_freq.append(float(temp))
                
                        self.cpu_type.append(temp0)
                    else:
                        temp0=re.split(r'(\d{1}GHz)',j)[0]
                        temp=re.search(r'(\d{1}GHz)',j).group().split('GHz')[0]
                        self.cpu_freq.append(float(temp))
                        self.cpu_type.append(temp0)
            elif i not in ["Ram",'Cpu',"Weight","ScreenResolution","Price_euros","Inches"]:
                self.dataset[i]=X.replace(to_replace=X.unique(),value=range(len(X.unique()))) 
        return self
    
    def transform(self,dataset):
        self.dataset.drop("Ram",axis=1,inplace=True)
        self.dataset["Ram(GB)"]=self.ram
        self.dataset.drop("Cpu",axis=1,inplace=True)
        self.dataset["Cpu_freq(GHz)"]=self.cpu_freq
        self.dataset["Cpu_type"]=self.cpu_type
        self.dataset["X_Res"]=self.x_res
        self.dataset["Y_Res"]=self.y_res
        self.dataset["Screen_type"]=self.scrn_type
        self.dataset.drop("ScreenResolution",axis=1,inplace=True)
        self.dataset["Screen_type"]   = self.dataset["Screen_type"].replace(to_replace= self.dataset["Screen_type"] .unique(),value=range(len( self.dataset["Screen_type"] .unique())))
        self.dataset["Cpu_type"]   = self.dataset["Cpu_type"].replace(to_replace= self.dataset["Cpu_type"] .unique(),value=range(len( self.dataset["Cpu_type"] .unique())))
        self.dataset.drop("Weight",axis=1,inplace=True)
        self.dataset["Weight(Kg)"]=self.wt
        return self.dataset
       

    
            

                   
       

    
            


In [2]:
data=pd.read_csv("laptop_price.csv",encoding='latin-1')
data.head(2)

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94


In [67]:
model=joblib.load('Finalmodel.pkl')

In [38]:
model

In [74]:
transformer=Transformer()
transformed_testx=transformer.fit_transform(data)

/tmp/ipykernel_1335042/247390603.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.dataset[i]=X.replace(to_replace=X.unique(),value=range(len(X.unique())))
/tmp/ipykernel_1335042/247390603.py:69: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.dataset[i]=X.replace(to_replace=X.unique(),value=range(len(X.unique())))
/tmp/ipykernel_1335042/247390603.py:82: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_object

In [75]:
features=data.drop("Price_euros",axis=1)
label=data["Price_euros"]

In [80]:
trainx,testx,trainy,testy=train_test_split(features,label,test_size=0.2)

In [81]:
trainy

599     1763.00
539      859.00
705      795.00
844     1870.00
1064    1929.00
         ...   
318      980.00
86       699.00
12      2439.97
817      385.00
1290     459.00
Name: Price_euros, Length: 1042, dtype: float64

In [82]:
predict=model.predict(testx)

In [86]:
from scipy import stats
squared_errors= (testy-predict)**2
confidence = 0.95
prederror=np.sqrt(stats.t.interval(confidence,len(squared_errors)-1,loc=squared_errors.mean(),scale=stats.sem(squared_errors))).mean()

In [84]:
max=testy.max()
min=testy.min()

In [88]:
percent=(1-prederror/(max-min))*100
print(f"Accuracy is {percent}%")

Accuracy is 92.11356685649235%
